## Allen-Cahn Equation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

### Data

In [ ]:
## Data
input_data = np.load("data/AC_data_input.npy")  # (1000, 1001, 2)
y_data = np.load("data/AC_data_output.npy")  # (1000, 1001)

x_data = input_data[:, :, 1]                 # (1000, 1001)
c_data = input_data[:, :, 0]                 # (1000, 1001)

In [ ]:
grid_size = 1001
n_train = 800
n_test = 200
mesh = np.linspace(-1, 1, grid_size)

x_train_np = x_data[:n_train]       # (800, 1001)
c_train_np = c_data[:n_train]       # (800, 1001)
y_train_np = y_data[:n_train]       # (800, 1001)

x_test_np = x_data[-n_test:]        # (200, 1001)
c_test_np = c_data[-n_test:]        # (200, 1001)
y_test_np = y_data[-n_test:]        # (200, 1001)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 32

x_train = torch.tensor(x_train_np).float()  # (800, 101)
c_train = torch.tensor(c_train_np).float()  # (800, 101)
y_train = torch.tensor(y_train_np).float()  # (800, 101)

x_test = torch.tensor(x_test_np).float()    # (200, 101)
c_test = torch.tensor(c_test_np).float()    # (200, 101)
y_test = torch.tensor(y_test_np).float()    # (200, 101)

train_loader = DataLoader(TensorDataset(x_train, c_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test, c_test, y_test), batch_size=batch_size, shuffle=False)

x, c, y = next(iter(train_loader))
print(">> Batch:", x.shape, c.shape, y.shape)

### Model

In [ ]:
class MLP(nn.Module):
    def __init__(self, layer_sizes, activation, final_activation=None):
        super().__init__()
        self.input_size = layer_sizes[0]
        self.output_size = layer_sizes[-1]

        layers = []
        for i in range(len(layer_sizes) - 2):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            layers.append(activation)
        layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))

        if final_activation is not None:
            layers.append(final_activation)

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    
class DeepOnet(nn.Module):
    def __init__(self, grid_size, width):
        super().__init__()
        self.branch_net = MLP(layer_sizes=[grid_size, 64, 64, 64, 64, 64, width], 
                              activation=nn.ReLU())
        self.trunk_net = MLP(layer_sizes=[1, 64, 64, 64, 64, 64, width], 
                             activation=nn.Tanh(), final_activation=nn.Tanh())

    def forward(self, x, c):
        out_branch = self.branch_net(x)
        out_trunk = self.trunk_net(c)
        out = torch.sum(out_branch * out_trunk, axis=-1)
        return out

### Training

In [ ]:
# Training
def preprocess(x, c, y=None, device=device):
    grid_size = x.shape[1]
    x = x[:, None, :].repeat(1, grid_size, 1)   # (N, grid_size) -> (N, grid_size, grid_size)
    c = c[:, :, None]                           # (N, grid_size) -> (N, grid_size, 1)
    x, c = x.to(device), c.to(device)
    return (x, c) if y is None else (x, c, y.to(device))

n_epochs = 200
learning_rate = 1e-3

set_seed(42)
model = DeepOnet(grid_size, width=64).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.95)

for epoch in range(1, n_epochs + 1):
    model.train()
    loss_mse = 0
    train_loss = 0
    for x, c, y in train_loader:
        x, c, y = preprocess(x, c, y)
        y_pred = model(x, c)
        loss = loss_fn(y_pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    model.eval()
    test_loss = 0
    with torch.no_grad():
        for x, c, y in test_loader:
            x, c, y = preprocess(x, c, y)
            y_pred = model(x, c)
            test_loss += loss_fn(y_pred, y).item()

    train_loss /= len(train_loader)
    test_loss /= len(test_loader)
    scheduler.step()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch:4d}/{n_epochs}] (lr: {scheduler.get_last_lr()[0]:.2e}) "
              f"train_loss: {train_loss:.2e} | test_loss: {test_loss:.2e}")

### Evaluation

In [ ]:
## Evaluation
model.eval()
with torch.no_grad():
    x, c = preprocess(x_test, c_test)
    y_pred = model(x, c)

y_pred_np = y_pred.cpu().detach().numpy()

idx = 10
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 3))
ax1.plot(c_test_np[idx], x_test_np[idx], 'b', label="Input")
ax1.plot(c_test_np[idx], y_test_np[idx], 'k:', label="Exact")
ax1.plot(c_test_np[idx], y_pred_np[idx], 'r', lw=1, label="Prediction")
ax2.plot(c_test_np[idx], y_test_np[idx] - y_pred_np[idx], 'k', label="Error")

for ax in (ax1, ax2):
    ax.legend(loc="upper right")
    ax.grid(color='k', ls=':', lw=1)
fig.tight_layout()
plt.show()